In [23]:
from neural_features import ResNet50FeatureExtractor
import numpy as np
import pandas as pd
from torchvision.models import resnet50
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import shutil
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from neighbor_distances import CentroidWeights
from sklearn.model_selection import train_test_split

In [14]:
def image_retrieval_using_knn(n_neighbors, dataset, queryset):
    pipeline = Pipeline([
        ('knn', KNeighborsClassifier(n_neighbors=n_neighbors))  
    ])

    pipeline.fit(dataset[0], dataset[1])
    predictions = pipeline.predict(queryset)

    return predictions
    

In [7]:
train_small_names = pd.read_csv('../dataset/train_small.csv', header=None)
train_unlabeled_names = pd.read_csv('../dataset/train_small.csv', header=None)

small_dataset = np.load('../dataset/small_train_features_resnet50.npz', 'r')
X = small_dataset['X']
X_label = small_dataset['y']

queryset = np.load('../dataset/train_unlabeled_features_resnet50_completed.npz', 'r')
Q = queryset['X']
Q_label = queryset['y']

In [10]:
extracted_indeces = np.random.choice(Q.shape[0], size=int(len(Q) * 0.10), replace=False)
print(extracted_indeces)


[ 20361  15882  66025 ...  96406  79065 110357]


In [36]:
Q_small = Q[extracted_indeces, :]
Q_small.shape

(11345, 2048)

In [15]:
predictions = image_retrieval_using_knn(5, (X, X_label), Q_small)


In [22]:
int((.2 * (Q_small.shape[0] + X.shape[0]) )/ 251)/20


0.65

In [39]:
X_train, X_test, y_train, y_test = train_test_split(X, X_label, test_size=int((.2 * (Q_small.shape[0] + X.shape[0]) )/ 251)/20, stratify=X_label, random_state=42)

In [40]:
X_train = np.concatenate((X_train, Q_small), axis=0)  # Combine rows
y_train = np.concatenate((y_train, predictions), axis=0)  # Combine

In [41]:
X_train.shape

(13102, 2048)

In [42]:
y_train.shape

(13102,)

In [43]:
X_test.shape

(3263, 2048)

In [44]:
y_test.shape

(3263,)

In [45]:
np.savez("../dataset/features_extended_10.npz", X_train = X_train, y_train = y_train, X_val=X_test, y_val=y_test)